# Summary Statistics 
2023-08-08 ZD  

This notebook will explore options to gather summary statistics and other reporting data calculated from data within processed grants data.  

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
from itertools import combinations


# Method to import from parent directory
import os
import sys
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(root_dir)

import config

### Load data from all Key Program grants output CSVs into a single dataframe for stats

In [2]:
# Define directory containing processed grant data
processed_dir = '../'+config.PROCESSED_DIR

# Define directory to store reports. Create if doesn't already exist
reports_dir = '../'+config.REPORTS_DIR
if not os.path.exists(reports_dir):
    os.makedirs(reports_dir)

# # Backup paths to allow for using same set of gathered data during notebook development
# processed_dir = '../' + 'data/processed/2023-07-19/api-gathered-2023-08-25'
# reports_dir = '../' + 'reports/2023-07-19/api-gathered-2023-08-25'
if not os.path.exists(reports_dir):
    os.makedirs(reports_dir)

print(f"Project data pulled from: {processed_dir}")
print(f"Reports will be output to: {reports_dir}")

Project data pulled from: ../data/processed/2023-08-30/api-gathered-2023-08-30
Reports will be output to: ../reports/2023-08-30/api-gathered-2023-08-30


In [3]:
# Load grants data
grants_filename = os.path.join(processed_dir, 'project.tsv')
df = pd.read_csv(grants_filename, sep='\t')

# Rename program.program_id column
df.rename(columns={'program.program_id':'program'}, inplace=True)

### Quick detour to handle abstract text cleaning

In [4]:
# Pull one abstract to check for odd character encoding
text = df.abstract_text[1]
text

'Project Summary - Overall The mortality rates of breast and pancreatic cancers are intrinsically tied to metastasis. In pancreatic cancer, the 5-year survival rate is only 9% and in ~70% pathological evaluations of the resected tumor, instances of venous invasion are found. Metastasis is a complex multi-step process involving cancer cells, local vasculature, and the surrounding microenvironment at multiple sites. Venous invasion in pancreatic cancer, in which cancer cells gain often invade the portal vein, is an early step in this process and provides the cells a direct path to the liver, the most common site of pancreatic cancer metastasis. As in pancreatic cancer, invasion past normal breast tissue barriers is critically tied to breast cancer outcomes. Most breast tumors can be surgically removed, and so mortality is closely tied to the extent of distant metastasis through lymphovascular invasion. The detection of lymphovascular invasion in a breast tumor correlates with poor progno

Lots of odd characters in here like "\xad" and "\xa0". Double-spaces were an issue before as well.  
Make a function that can clean the column with regex and space replacement.


In [5]:
def clean_abstract(text):
    # Define a list of characters to be removed or replaced
    chars_to_remove = ['\xad']
    chars_to_space = ['  ']

    if isinstance(text, str):
        # Remove unwanted characters (no space added)
        for char in chars_to_remove:
            text = text.replace(char, '')
        # Replace unwanted characters with a space
        for char in chars_to_space:
            text = text.replace(char, ' ')

        # Remove non-breaking spaces, newlines, and other unwanted characters
        cleaned_text = re.sub(r'[\s\xa0]+', ' ', text).strip()
        return cleaned_text
        
    # Return original value if it is NaN or float
    else: 
        return text

In [6]:
# Try it on the example abstract from before
clean = clean_abstract(text)
clean

'Project Summary - Overall The mortality rates of breast and pancreatic cancers are intrinsically tied to metastasis. In pancreatic cancer, the 5-year survival rate is only 9% and in ~70% pathological evaluations of the resected tumor, instances of venous invasion are found. Metastasis is a complex multi-step process involving cancer cells, local vasculature, and the surrounding microenvironment at multiple sites. Venous invasion in pancreatic cancer, in which cancer cells gain often invade the portal vein, is an early step in this process and provides the cells a direct path to the liver, the most common site of pancreatic cancer metastasis. As in pancreatic cancer, invasion past normal breast tissue barriers is critically tied to breast cancer outcomes. Most breast tumors can be surgically removed, and so mortality is closely tied to the extent of distant metastasis through lymphovascular invasion. The detection of lymphovascular invasion in a breast tumor correlates with poor progno

In [7]:
# Clean abstract column
df['abstract_text'] = df['abstract_text'].apply(clean_abstract)

In [8]:
# Check top 10 abstracts 
for abstract in df['abstract_text'][0:10]: print(abstract)

This project proposes an integrated suite of microscopy and data analysis advances that would enable quantitative, mechanistic analysis of immune-microenvironment dynamics in poor prognosis solid tumors. While immunotherapies are showing remarkable clinical responses in some advanced cancers, to date, their impact on many solid tumors has been modest. This is due, in part, to solid tumor microenvironments limiting the effectiveness of natural immune responses and immunotherapies. Yet, our understanding of the physical and molecular mechanisms governing T cell infiltration, distribution, and function in native tumor microenvironments remains extremely limited. As such, defining key T cell behaviors as a function of complex tumor microenvironments will identify design criteria that can be used to develop novel cell engineering strategies that optimize T cell-centric therapies for solid tumors. To this end the research theme of the U54 Center for Multiparametric Imaging of Tumor Immune Mi

No sign of the problem characters. There is likely a better way to clean this in a less explicitly defined way, but this works well enough to get rid of the egregious encoding oddities and can be expanded as others are identified.

### Start exploring for patterns and stats to report out

In [9]:
# Look at a single row in detail
df.loc[0]

project_num                                                  5U54CA268069-02
core_project_num                                                 U54CA268069
appl_id                                                             10538588
fiscal_year                                                             2023
project_title              Center for Multiparametric Imaging of Tumor Im...
abstract_text              This project proposes an integrated suite of m...
pref_terms                 Address;Advanced Malignant Neoplasm;Behavior;B...
org_name                                             UNIVERSITY OF MINNESOTA
org_city                                                         MINNEAPOLIS
org_state                                                                 MN
org_country                                                    UNITED STATES
principal_investigators             Kevin William Eliceiri, Paolo Provenzano
program_officers                                               Steven Becker

In [10]:
df.dtypes

project_num                object
core_project_num           object
appl_id                     int64
fiscal_year                 int64
project_title              object
abstract_text              object
pref_terms                 object
org_name                   object
org_city                   object
org_state                  object
org_country                object
principal_investigators    object
program_officers           object
award_amount                int64
agency_ic_fundings          int64
award_notice_date          object
project_start_date         object
project_end_date           object
full_foa                   object
api_source_search          object
program                    object
dtype: object

In [11]:
# Check fiscal years of all grants
df['fiscal_year'].value_counts().reset_index().sort_values(by='index')

,index,fiscal_year
23,2000,8
21,2001,10
22,2002,8
20,2003,11
15,2004,14
11,2005,19
14,2006,15
17,2007,13
18,2008,12
13,2009,16


In [12]:
# Check columns
df.columns.tolist()

['project_num',
 'core_project_num',
 'appl_id',
 'fiscal_year',
 'project_title',
 'abstract_text',
 'pref_terms',
 'org_name',
 'org_city',
 'org_state',
 'org_country',
 'principal_investigators',
 'program_officers',
 'award_amount',
 'agency_ic_fundings',
 'award_notice_date',
 'project_start_date',
 'project_end_date',
 'full_foa',
 'api_source_search',
 'program']

In [13]:
# Get number of core projects for each program
df.groupby('program')['core_project_num'].nunique().reset_index()

,program,core_project_num
0,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,1
1,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...,56
2,ADataResourceforBloodandMarrowTransplantsandAd...,1
3,AIDSandCancerSpecimenResourceACSR,1
4,AcceleratingColorectalCancerScreeningandfollow...,11
...,...,...
63,TheUniversityofTexasMDAndersonCancerCenterSPOR...,1
64,TranslationalandBasicScienceResearchinEarlyLes...,6
65,UniversityofTexasSPOREinLungCancer,1
66,VanderbiltIngramCancerCenterSPOREinGastrointes...,1


### Build a summary of all Programs and the number of projects for each

In [14]:
# Get the year from the date objects
# Lose some resolution but make the data easier to read for stats
df['project_start_date_year'] = df['project_start_date'].apply(lambda x: int(x[:4]))
df['project_end_date_year'] = df['project_end_date'].apply(lambda x: int(x[:4]))

In [15]:
# Copy fiscal year column for later min and max stats
df['fiscal_year_copy'] = df['fiscal_year']

In [16]:
# Define functions to apply to each column
agg_funcs = {
    'api_source_search': 'nunique',
    'core_project_num': 'nunique',
    'project_num': 'nunique',
    'agency_ic_fundings': 'sum', # This will be a little off due to duplicates!
    'project_start_date_year': 'min',
    'project_end_date_year': 'max',
    'fiscal_year': 'min',
    'fiscal_year_copy': 'max'
}

In [17]:
# # Define column titles better suited for reporting
# rename_dict = {
#     "program": "Program",
#     "core_project_num": "Core Project Count",
#     "project_num": "Grant/Award Count",
#     'agency_ic_fundings': "Total NCI Funding (since 2000)", # This will be a little off due to duplicates! 
#     "api_source_search": "Provided NOFOs/Awards with Associated Grants",
#     "project_start_date_year": "Earliest Project Start Date",
#     "project_end_date_year": "Latest Project End Date",
#     "fiscal_year": "Earliest Fiscal Year",
#     "fiscal_year_copy": "Latest Fiscal Year"
# }

In [18]:
# Group by 'program' and apply aggregation functions defined above
summary_stat_df = df.groupby('program').agg(agg_funcs).reset_index()

# # Rename columns for better presentation as defined above
# summary_stat_df.rename(columns=rename_dict, inplace=True)

# # Store program summary 
# program_summary_filename = reports_dir + '/' + 'programSummaryStats.csv'
# summary_stat_df.to_csv(program_summary_filename, index=False)

In [19]:
summary_stat_df

,program,api_source_search,core_project_num,project_num,agency_ic_fundings,project_start_date_year,project_end_date_year,fiscal_year,fiscal_year_copy
0,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,1,1,4,2107456,2020,2023,2020,2022
1,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...,4,56,187,100883313,2013,2024,2016,2023
2,ADataResourceforBloodandMarrowTransplantsandAd...,3,1,12,28797489,1998,2028,2013,2023
3,AIDSandCancerSpecimenResourceACSR,1,1,4,18189601,2013,2024,2019,2022
4,AcceleratingColorectalCancerScreeningandfollow...,3,11,26,27008270,2018,2024,2018,2022
...,...,...,...,...,...,...,...,...,...
63,TheUniversityofTexasMDAndersonCancerCenterSPOR...,1,1,4,9063325,2019,2024,2019,2022
64,TranslationalandBasicScienceResearchinEarlyLes...,2,6,6,9095020,2022,2027,2022,2022
65,UniversityofTexasSPOREinLungCancer,1,1,35,52951102,1996,2025,2000,2023
66,VanderbiltIngramCancerCenterSPOREinGastrointes...,1,1,5,11590546,2019,2024,2019,2023


For the most part, Earliest Grant Date aligns with earliest Fiscal Year. Outliers are the very early start dates for All of Us, ALCHEMIST, and EDRN programs compared to the earlist fiscal year. This might indicate a project that received additional funds or supplements many years after the proejct began. Could this be data deposition or a similar modernization effort?

## Enrich Key Program CSV with Grants data
This will begin to mock up the INS Key Programs page as requested by ODS. 

In [20]:
# Use config to get path and load clean key programs csv 
key_programs_filename = '../' + config.CLEANED_KEY_PROGRAMS_CSV
key_programs_df = pd.read_csv(key_programs_filename)

In [21]:
# Make a temporary program ID column to use for mapping stats to key programs
key_programs_df['program'] = key_programs_df['program_name'].apply(lambda name: ''.join(filter(str.isalnum, name)))

In [22]:
# Check columns in key programs df
key_programs_df.columns.tolist()

['program_name',
 'program_acronym',
 'focus_area',
 'doc',
 'contact_pi',
 'contact_pi_email',
 'contact_nih',
 'contact_nih_email',
 'nofo',
 'award',
 'program_link',
 'data_link',
 'cancer_type',
 'program']

In [23]:
# Check columns in program summary stats df
summary_stat_df.columns.tolist()

['program',
 'api_source_search',
 'core_project_num',
 'project_num',
 'agency_ic_fundings',
 'project_start_date_year',
 'project_end_date_year',
 'fiscal_year',
 'fiscal_year_copy']

In [24]:
# Add stats to key programs df using temp program ID as connector
key_programs_enriched = pd.merge(left=key_programs_df, 
                                right=summary_stat_df[['program',
                                                       'core_project_num', 
                                                       'project_num',
                                                       'agency_ic_fundings']], 
                                on='program')
# Drop temp program ID column
key_programs_enriched.drop(columns='program', inplace=True)

# Rename stat columns 
stat_renamer_dict = {
    'core_project_num': 'core_project_count',
    'project_num': 'grant_count',
    'agency_ic_fundings': 'total_nci_funding',
}
key_programs_enriched.rename(columns=stat_renamer_dict, inplace=True)

In [25]:
key_programs_enriched.columns.tolist()

['program_name',
 'program_acronym',
 'focus_area',
 'doc',
 'contact_pi',
 'contact_pi_email',
 'contact_nih',
 'contact_nih_email',
 'nofo',
 'award',
 'program_link',
 'data_link',
 'cancer_type',
 'core_project_count',
 'grant_count',
 'total_nci_funding']

In [26]:
# Store enriched Key Programs Stats Table
key_program_stats_filename = reports_dir + '/' + 'keyProgramStats.csv'
key_programs_enriched.to_csv(key_program_stats_filename, index=False)

## Continue exploring data report options

In [27]:
# Get a df of all grants with project start dates before year 2000
early_project_df = df[df['project_start_date_year'] < 2000]

In [28]:
# # Export to csv for quick ad-hoc analysis
# early_project_df.to_csv('earlyProjectReport.csv',index=False)

In [29]:
# Group to find patterns and common sources for the projects with very early start dates
early_project_df.groupby(['program', 'api_source_search', 'core_project_num', 'project_title'])['project_num'].nunique().reset_index()

,program,api_source_search,core_project_num,project_title,project_num
0,ADataResourceforBloodandMarrowTransplantsandAd...,nofo_CA-12-503,U24CA076518,A Data Resource for Analyzing Blood and Marrow...,6
1,ADataResourceforBloodandMarrowTransplantsandAd...,nofo_CA-17-031,U24CA076518,A Data Resource for Analyzing Blood and Marrow...,5
2,ADataResourceforBloodandMarrowTransplantsandAd...,nofo_CA-22-026,U24CA076518,A Data Resource for Blood and Marrow Transplan...,1
3,AllofUs,nofo_PA20-185,R01CA031845,Synthetic Studies Related to Cancer Research/T...,2
4,AllofUs,nofo_PA20-185,R01CA047296,A Pathway of Tumor Suppression,2
...,...,...,...,...,...
115,UniversityofTexasSPOREinLungCancer,award_P50CA070907,P50CA070907,SPORE in Lung Cancer,8
116,UniversityofTexasSPOREinLungCancer,award_P50CA070907,P50CA070907,"SPORE: Developing New Rationale, Personalized ...",7
117,UniversityofTexasSPOREinLungCancer,award_P50CA070907,P50CA070907,Targeting Lung Cancer Vulnerabilities,3
118,UniversityofTexasSPOREinLungCancer,award_P50CA070907,P50CA070907,UNIVERSITY OF TEXAS SPORE IN LUNG CANCER,10


### Check for "Bottleneck Effect" grants

In [30]:
def find_rows_with_different_values(df, shared_column, compare_column):
    """Find rows that share a value in a specified column
    but have different values in another specified column.
    """

    grouped = df.groupby(shared_column)[compare_column].transform('nunique')
    selected_row_df = df[grouped > 1]

    return selected_row_df

In [31]:
# Check for Core Projects found in multiple programs and compare sources

shared_column = 'core_project_num'
compare_column = 'program'

# Get grant-level rows with same project but different program
df_shared = find_rows_with_different_values(df, shared_column, compare_column)

# Group with the provided search value and count unique grants 
df_shared_projects = (df_shared.groupby(
                        ['api_source_search', shared_column, compare_column])
                        .size().reset_index()
                        .rename(columns={0:'grant_count'}))

# Store shared programs
shared_projects_filename = reports_dir + '/' + 'sharedProjects.csv'
df_shared_projects.to_csv(shared_projects_filename, index=False)

For any Core Project gathered for more than one program (e.g. shared NOFOs or a NOFO in one program and an Award in another), the grants and all downstream outputs will need to be associated with BOTH programs. It's important to identify these. 

In [32]:
# Show core projects found in multiple programs
df_shared_projects

,api_source_search,core_project_num,program,grant_count
0,award_P50CA098131,P50CA098131,SPOREinBreastCancer,26
1,award_R01CA239701,R01CA239701,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,4
2,award_U10CA180821,U10CA180821,TheAdjuvantLungCancerEnrichmentMarkerIdentific...,35
3,nofo_CA-17-015,U01CA224145,CONSORTIUMFORPANCREATICDUCTALADENOCARCINOMAPDA...,3
4,nofo_CA-17-015,U01CA224146,CONSORTIUMFORPANCREATICDUCTALADENOCARCINOMAPDA...,3
...,...,...,...,...
116,nofo_RFA-CA-22-038,U24CA224319,CANCERIMMUNEMONITORINGANDANALYSISCENTERS,1
117,nofo_RFA-CA-22-038,U24CA224319,CancerImmunologicDataCommons,1
118,nofo_RFA-CA-22-038,U24CA224331,CANCERIMMUNEMONITORINGANDANALYSISCENTERS,1
119,nofo_RFA-CA-22-038,U24CA224331,CancerImmunologicDataCommons,1


Some Key Programs specify Awards for grants rather than the entire NOFO. Only the Award and downstream grants should be associated with that Program, rather than everything dowstream of the larger NOFO.  
However, all grants gathered from NIH RePORTER necessarily have a 'full_foa' (aka NOFO) value within the RePORTER data. This is a potential for complications.  

We need to check for two things:  
1. Did we search NIH RePORTER with any NOFOs that do not match the full_foa within grants returned? 
    - That would be unexpected and indicate an API gathering problem.
    - Would appear in the table below as a rows with "nofo_"... in the api_source_search column
2. Did we search NIH RePORTER with any Awards with differing full_foa values within grants returned? 
    - That's acceptable - it could indicate that the particular Core Project received funding from many sources (e.g. data sharing NOFO effort)
    - It's still important to note. We need to be careful not to accidentally make "upstream" connections from grants back to full_foas back to programs for projects derived from specified Award gathering.


In [33]:
# Group by core project and combine programs into unordered set
grouped = df.groupby("core_project_num")["program"].apply(set)

# Create combos of programs for each core project with itertools combinations
df_shared_programs = grouped.apply(lambda x: list(combinations(x, 2)))

# Flatten the program combos and count occurrences
df_shared_programs = df_shared_programs.explode().value_counts().reset_index()

# Split program combo column into two separate columns
# NOTE this will need to be reworked if a project has more than 2 programs
df_shared_programs[['program_1','program_2']] = df_shared_programs['index'].apply(pd.Series)

# Reorder and rename
df_shared_programs.rename(columns={'program':'shared_project_count'}, inplace=True)
df_shared_programs = df_shared_programs[['program_1','program_2','shared_project_count']]

# Export as report
shared_programs_filename = reports_dir + '/' + 'sharedProjectsByProgramPair.csv'
df_shared_programs.to_csv(shared_programs_filename, index=False)

# Show table of program combos and number of shared projects between them
df_shared_programs

,program_1,program_2,shared_project_count
0,OncologyModelsForumU24,AllofUs,6
1,CONSORTIUMFORPANCREATICDUCTALADENOCARCINOMAPDA...,PancreaticCancerMicroenvironmentNetwork,6
2,CancerSystemsBiologyConsortiumCSBC,AllofUs,6
3,AllofUs,PediatricPreclinicalinVivoTestingPIVOT,5
4,FusionOncoproteinsinChildhoodCancersFusOnc2,AllofUs,5
5,HumanTumorAtlasNetwork,AllofUs,5
6,CancerImmunologicDataCommons,CANCERIMMUNEMONITORINGANDANALYSISCENTERS,4
7,AllofUs,BarrettsEsophagusTranslationalResearchNetworkB...,4
8,HIVAssociatedMalignancyResearchCenters,AllofUs,3
9,AllofUs,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...,2


In [34]:
# Check for rows with the same ODS-provided NOFO but different FOA/NOFO accordign to RePORTER 
shared_column = 'api_source_search'
compare_column = 'full_foa'

# Select only rows with different values
df_shared = find_rows_with_different_values(df, shared_column, compare_column)
# Group and summarize
df_shared.groupby([shared_column, compare_column]).size().reset_index().rename(columns={0:'grant_count'})

,api_source_search,full_foa,grant_count
0,award_P50CA058223,PAR-00-087,10
1,award_P50CA058223,PAR-05-156,8
2,award_P50CA058223,PAR-10-003,7
3,award_P50CA058223,PAR-14-353,5
4,award_P50CA058223,RFA-CA-94-027,2
...,...,...,...
61,award_R01CA239701,PA-20-272,2
62,award_R01CA239701,PA-21-071,1
63,award_U10CA180821,PA-20-272,8
64,award_U10CA180821,RFA-CA-12-010,16


## Build visualizations in Plotly
The goal is to communicate patterns and highlight limitations or oddities within the grants data in order to provide feedback and capabilities to ODS.  
Visualizations may be a better route than spreadsheets to summarize this. 

### Program Funding

#### Pie Chart

In [35]:
# # Grouping data to calculate the sum of total_cost for each program
# program_agency_funding = df.groupby('program')['agency_ic_fundings'].sum().reset_index()

# # Creating the Pie Chart
# fig = px.pie(program_agency_funding, 
#              names='program', 
#              values='agency_ic_fundings', 
#              title='Distribution of Agency IC Fundings Across Programs',
#              height=600,
#              width=1200,
#              )

# # Show the Pie Chart
# fig.show()

#### TreeMap Block Chart

In [36]:
# # Grouping data to calculate the sum of total_cost for each program
# # program_core_funding = df.groupby(['program', 'core_project_num'])['agency_ic_fundings'].sum().reset_index()

# # Creating the Treemap
# fig = px.treemap(df, 
#                  path=['program', 'project_title', 'project_num'], 
#                  values='agency_ic_fundings',
#                  title='Distribution of NCI Funding by Program, Project, and Award',
#                  height=1200,
#                  width=1800)

# # Show the Treemap
# fig.show()

In [37]:
# Grouping data to calculate the sum of agency_ic_funding for each org_state
state_funding = df.groupby('org_state', dropna=False)['agency_ic_fundings'].sum().reset_index()
state_funding

,org_state,agency_ic_fundings
0,AL,12158908
1,AR,10446810
2,AZ,41317184
3,BC,3288053
4,CA,450768605
5,CO,16094052
6,CT,68695568
7,DC,40861789
8,DE,823703
9,FL,54670242


#### Chloropleth (US Map with funding by state)

In [38]:
# # Grouping data to calculate the sum of agency_ic_funding for each org_state
# state_funding = df.groupby('org_state',dropna=False)['agency_ic_fundings'].sum().reset_index()

# # Creating the Chloropleth
# fig = px.choropleth(state_funding, 
#                     locations='org_state', 
#                     locationmode='USA-states', 
#                     color='agency_ic_fundings',
#                     scope='usa',
#                     color_continuous_scale='Blues',
#                     title='Total NCI Funding by State since 2000',
#                     height=600,
#                     width=1200)

# # Show the Chloropleth
# fig.show()

In [39]:
program_fy_funding = df.groupby(['program','fiscal_year'])['agency_ic_fundings'].sum().reset_index()
program_fy_funding

,program,fiscal_year,agency_ic_fundings
0,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,2020,1898254
1,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,2021,11554
2,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,2022,197648
3,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...,2016,3167336
4,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...,2017,12039602
...,...,...,...
470,YaleSPOREinLungCancerYSILCTheBiologyandPersona...,2019,2134771
471,YaleSPOREinLungCancerYSILCTheBiologyandPersona...,2020,2207890
472,YaleSPOREinLungCancerYSILCTheBiologyandPersona...,2021,2003668
473,YaleSPOREinLungCancerYSILCTheBiologyandPersona...,2022,2046999


#### Stacked Bar Chart (Program funding per year)

In [40]:
# # Group data
# program_fy_funding = df.groupby(['program','fiscal_year'])['agency_ic_fundings'].sum().reset_index()

# # Create the line chart
# fig = px.bar(program_fy_funding, 
#               x='fiscal_year', 
#               y='agency_ic_fundings', 
#               color='program', 
#               title='NCI Funding Since Fiscal Year 2000 by Key Program',
#               labels={'fiscal_year': 'Fiscal Year', 'agency_ic_fundings': 'NCI Funding', 'program':'Key Program'},
#               height=600,
#               width=1800
#             )
# # Show the line chart
# fig.show()

#### Sankey Diagram
There was a lot of trial and error not fully shown here

In [41]:
# Sum NCI funding for each program, NOFO, and project combo
sankey_summary = df.groupby(['program', 'full_foa', 'core_project_num'])['agency_ic_fundings'].sum().reset_index()
sankey_summary

,program,full_foa,core_project_num,agency_ic_fundings
0,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,PA-19-056,R01CA239701,1898254
1,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,PA-20-272,R01CA239701,197648
2,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,PA-21-071,R01CA239701,11554
3,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...,PAR-15-331,U24CA180922,3997961
4,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...,PAR-15-331,U24CA180996,2099900
...,...,...,...,...
2408,UniversityofTexasSPOREinLungCancer,PAR-18-313,P50CA070907,6449069
2409,VanderbiltIngramCancerCenterSPOREinGastrointes...,PAR-18-313,P50CA236733,11590546
2410,YaleSPOREinLungCancerYSILCTheBiologyandPersona...,PAR-14-031,P50CA196530,12004456
2411,YaleSPOREinLungCancerYSILCTheBiologyandPersona...,PAR-18-313,P50CA196530,8328403


In [42]:
# Build connections from programs to full_foas
links_program_nofo = df.groupby(['program','full_foa'])['agency_ic_fundings'].sum().reset_index()
# Rename with standard cols
links_program_nofo.columns = ['source','target','value']


# Build connections from full_foas to project_nums
links_nofo_project = df.groupby(['full_foa','core_project_num'])['agency_ic_fundings'].sum().reset_index()
# Rename with standard cols
links_nofo_project.columns = ['source','target','value']

# Combine different links dataframes
links = pd.concat([links_program_nofo, links_nofo_project])

# Remove links with no funding
links = links[links['value'] > 0]
links

,source,target,value
0,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,PA-19-056,1898254
1,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,PA-20-272,197648
2,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,PA-21-071,11554
3,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...,PAR-15-331,49498005
4,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...,PAR-15-332,19027032
...,...,...,...
2377,RFA-CA-22-051,UG1CA284884,324621
2378,RFA-CA-22-051,UG1CA284918,275910
2379,RFA-CA-94-027,P50CA058223,1011600
2380,RFA-CA-95-021,P50CA062924,6538584


In [43]:
# Create nodes dataframe from links dataframe
nodes_data = pd.concat([links['source'], links['target']]).unique()
nodes = pd.DataFrame({
    'node': nodes_data,
    'node_id': range(len(nodes_data))
})

# Create a dictionary to map node names to node IDs
node_id_mapping = dict(zip(nodes['node'], nodes['node_id']))

# Map the node names to their corresponding node IDs in the links dataframe
links['source'] = links['source'].map(node_id_mapping)
links['target'] = links['target'].map(node_id_mapping)

links

,source,target,value
0,0,71,1898254
1,0,73,197648
2,0,74,11554
3,1,88,49498005
4,1,89,19027032
...,...,...,...
2377,181,2384,324621
2378,181,2385,275910
2379,182,1999,1011600
2380,183,2000,6538584


In [44]:
# # Create Sankey diagram
# fig = go.Figure(go.Sankey(
#     node=dict(
#         pad=15,
#         thickness=20,
#         line=dict(color="black", width=0.5),
#         label=nodes['node'],  # Use the node names as labels

#     ),
#     link=dict(
#         source=links['source'],
#         target=links['target'],
#         value=links['value']
#     )
# ))

# # Customize layout
# fig.update_layout(
#     title_text="NCI Key Program Funding Flow",
#     font_size=14,
#     height=1800,
#     width=1200
# )

# # Display the figure
# fig.show()

## Program Keywords (from Grants)

In [45]:
# Get top keywords across all grants gathered
df['pref_terms'].str.split(';').explode().reset_index().groupby('pref_terms').size().reset_index().sort_values(by=0, ascending=False).rename(columns={0:'keyword_count'}).head(50)

,pref_terms,keyword_count
5259,Malignant Neoplasms,3826
6558,Patients,3717
2413,Data,3498
13785,tumor,3320
12252,novel,3283
8541,Testing,3231
3775,Goals,3089
1851,Clinical,2996
2531,Development,2923
11337,improved,2909


In [46]:
# Split keywords and explode
keywords_df = df.assign(pref_terms=df['pref_terms'].str.split(';')).explode('pref_terms')

# Count keyword frequencies per program
keyword_counts = keywords_df.groupby(['program', 'pref_terms']).size().reset_index(name='keyword_count')

keyword_counts

,program,pref_terms,keyword_count
0,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,12 year old,1
1,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,Accounting,1
2,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,Acute Lymphocytic Leukemia,4
3,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,Administrative Supplement,2
4,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...,Admixture,4
...,...,...,...
40416,YaleSPOREinLungCancerYSILCTheBiologyandPersona...,tumor,11
40417,YaleSPOREinLungCancerYSILCTheBiologyandPersona...,tumor initiation,1
40418,YaleSPOREinLungCancerYSILCTheBiologyandPersona...,tumor microenvironment,4
40419,YaleSPOREinLungCancerYSILCTheBiologyandPersona...,tumor progression,11


Keywords seem valuable but it's difficult to find an appropriate visualization. I'd rather not rely on WordCloud. 